In [24]:
import requests
from bs4 import BeautifulSoup
import urllib.request
import nltk
import re
import pandas as pd

In [25]:
# query = "Budweiser"
def get_url(query):
    page = requests.get("https://www.google.com/search?q="+query+"+alcohol+wikipedia&oq="+query+"+alcohol+wikiped&aqs=chrome.1.69i57j33.11038j1j7&sourceid=chrome&ie=UTF-8")
    soup = BeautifulSoup(page.content)
    urlstr=""
    for link in soup.find_all("a",href=re.compile("(?<=/url\?q=)(htt.*://.*)")):
        url = re.split(":(?=http)",link["href"].replace("/url?q=",""))
        urlstr = ' '.join(map(str, url)) 
        urlstr = urlstr.split('&')[0]
        break
    return urlstr

In [26]:
def list_to_string(s):  
    str1 = " " 
    return (str1.join(s)) 

In [27]:
def open_url(urlstr):
    page = requests.get(urlstr)
    soup = BeautifulSoup(page.content)

    info= ""
    whitelist = [
      'p','b','a'
    ]

    text_elements = [t for t in soup.find_all(text=True) if t.parent.name in whitelist]

    y=list_to_string(text_elements)
#     print(y)
    return y

In [28]:
def search(word, sentences):
       return [i for i in sentences if re.search(r'\b%s\b' % word, i)]

In [29]:
def get_content(y):
    s = re.split(r'[.?!:]+', y)

    a=search('is a', s)
    a+=search('is an ', s)

    content = list_to_string(a)
    return content

In [37]:
def check_name(query):
    type_list = ['vodka','gin','baijiu','tequila','rum','whisky','brandy','singani','soju','beer','wine','cider']
    query = query.lower()
    for item in type_list:
        if(query.find(item)!= -1):
            break
    return item

def get_type(content): 
    type_list = ['vodka','gin','baijiu','tequila','rum','whisky','brandy','singani','soju','beer','wine','cider']
    content_lower = content.lower()
    for item in type_list:
        if(content_lower.find(item)!= -1):
            break
    return item

           
def get_sub_type(content):
    beer = ['brown ale','lager','pale lager','pale ale','india pale ale (IPA)','porter stout','belgian style beer','wheat beer',
            'pale lagers and pilsners','dark lagers','german-style bocks','american sour','belgian fruit lambic',
            'flanders red ale','belgian gueuze']
    
    vodka = ['absolut','belvedere','grey goose','smirnoff','stolichnaya','three olives','uv vodka','van gogh']
    
    gin = ['london dry gin','plymouth gin','genever or dutch gin','old tom gin','new american or international style gin','sloe gin']
    
    tequila = ['blanco','joven','reposado','añejo','extra añejo']
    
    wine = ['red wine','rose wine','dessert wine','sweet wine','sparkling wine']
    
    rum = ['white or clear rum','gold or pale rum','dark rum','black rum','navy rum','premium aged rum','vintage rum','overproof rum',
           'rhum agricole','cachaça','aguardiente','flavored and spiced rum']
    
    whiskey = ['scotch','bourbon','irish whiskey','canadian whiskey','japanese whiskey']
    
    brandy = ['Cognac Brandy','Armagnac Brandy','Calvados','Brandy de Jerez','Obstler brandy','Pisco brandy','Armenian brandy',
             'Cypriot brandy','Pomace brandy','south african brandy']
    
    content_lower1 = content.lower()
    sub_type_of_alcohol = ""
    for a,b,c,d,e,f,g,h in zip(beer,vodka,gin,tequila,wine,rum,whiskey,brandy):
        if(content_lower1.find(a)!= -1):
            sub_type_of_alcohol = a
        elif(content_lower1.find(b)!= -1):
            sub_type_of_alcohol = b
        elif(content_lower1.find(c)!= -1):
            sub_type_of_alcohol = c
        elif(content_lower1.find(d)!= -1):
            sub_type_of_alcohol = d
        elif(content_lower1.find(e)!= -1):
            sub_type_of_alcohol = e    
        elif(content_lower1.find(f)!= -1):
            sub_type_of_alcohol = f    
        elif(content_lower1.find(g)!= -1):
            sub_type_of_alcohol = g    
        elif(content_lower1.find(h)!= -1):
            sub_type_of_alcohol = h  
        
    return sub_type_of_alcohol


In [31]:
def get_manufacturer(query,al_type):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36'}
    r = requests.get("https://www.google.com/search?biw=1280&bih=610&ei=e8RIXt7HCOK1mgfXy5PwBw&q="+query+al_type+"+manufacturer&oq="+query+al_type+"+manufacturer&gs_l=psy-ab.3..0i71l8.4032.4032..4220...0.2..0.0.0.......0....1..gws-wiz.pkv0d6RvK9Q&ved=0ahUKEwie8b_AndXnAhXimuYKHdflBH4Q4dUDCAs&uact=5", headers=headers)

#     r = requests.get("https://www.google.com/search?q="+query+"+alcohol+manufacturer&rlz=1C1CHBF_enIN880IN880&oq="+query+"+alcohol+manu&aqs=chrome.5.69i57j33l6.12357j0j7&sourceid=chrome&ie=UTF-8")
    soup = BeautifulSoup(r.text, 'lxml')
    #AZCkJd
    result = soup.find('div', class_='Z0LcW AZCkJd')
    if(result == None):
        result = soup.find('div', class_='Z0LcW')
        return (result.text)
    else: 
        return (result.text)

In [32]:
def get_alco_content(query):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36'}
    r = requests.get("https://www.google.com/search?safe=active&sxsrf=ALeKk01tNMV7-s3HnEgklgrrDXTbsTqEOw%3A1585549593397&ei=GZGBXrzqF9qF4-EPq665uAs&q="+query+"+alcohol+content&oq="+query+"+alcohol+content&gs_lcp=CgZwc3ktYWIQAzoECAAQRzoECAAQDToICAAQCBANEB46BQgAEM0CUJxNWIddYNhfaABwA3gAgAGqAYgBuwiSAQMwLjeYAQCgAQGqAQdnd3Mtd2l6&sclient=psy-ab&ved=0ahUKEwj82qbCyMHoAhXawjgGHStXDrcQ4dUDCAs&uact=5",headers=headers)
    soup = BeautifulSoup(r.text, 'lxml')
    #AZCkJd
    result = soup.find('div', class_='Z0LcW AZCkJd')
    if(result == None):
        result = soup.find('div', class_='Z0LcW')
        return(result.text)
    else: 
        return(result.text)

In [38]:
def driver(query):
    alco_type = check_name(query)
    if(alco_type == None):
        alcohol_type = get_type(content)
    else:
        alcohol_type = alco_type
    urlstr = get_url(query)
    y = open_url(urlstr)
    content = get_content(y)
    
    print("Alcohol type: "+alcohol_type)
    sub_type = get_sub_type(content)
    print("Alcohol subtype: "+sub_type)
    manufacturer = get_manufacturer(query,alcohol_type)
    print("Manufacturer: "+manufacturer)
    alco_content = get_alco_content(query)
    print("Alcohol content: "+alco_content)

In [39]:
driver("kingfisher premium lager beer")

Alcohol type: beer
Alcohol subtype: lager
Manufacturer: United Breweries - UB Group
Alcohol content: 4.8%
